In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import socket
import requests
import json
import urllib

In [2]:
addresses=open('dests.txt', 'r')
addresses=addresses.read()
addresses=addresses.split()

In [3]:
def geo(address,exp):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                lat=response_data['results'][0]['geometry']['location']['lat']
                lng=response_data['results'][0]['geometry']['location']['lng']
                if exp=='Longitude':
                    return lng
                else:
                    return lat
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

In [4]:
def add(address,exp):
    Serviceurl ='https://maps.googleapis.com/maps/api/distancematrix/json?'
    parms = dict()
    parms['origins']='Tel Aviv'
    parms['destinations'] = address
    parms['key'] = api_key
    url = Serviceurl + urllib.parse.urlencode(parms)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                dic_km=response_data['rows'][0]['elements'][0]['distance']['text'].split(' ')
                time_min=response_data['rows'][0]['elements'][0]['duration']['value']/60
                if exp=='time':
                    return round(time_min,2)
                else:
                    return dic_km[0]
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

In [5]:
df = pd.DataFrame(columns=('Target','Distance_km','Duration(hour+minutes)','Longitude','latitude'))                   

In [6]:
df['Target']=addresses
df['Distance_km']=df['Target'].apply(lambda x: add(x,'dis'))
df['Duration(hour+minutes)']=df['Target'].apply(lambda x: add(x,'time'))
df['Longitude']=df['Target'].apply(lambda x: geo(x,'Longitude'))
df['latitude']=df['Target'].apply(lambda x: geo(x,'latitude'))
print(df)

      Target Distance_km  Duration(hour+minutes)  Longitude   latitude
0   Istanbul       1,815                 1262.87  28.978359  41.008238
1  Amsterdam       4,533                 2883.48   4.904139  52.367573
2   Valletta       3,793                 3050.93  14.514100  35.899237
3      Basel       4,093                 2642.45   7.588576  47.559599
4       Doha       2,164                 1358.78  51.531040  25.285447


In [7]:
df_furthest=df.sort_values(by='Distance_km', ascending=False)
print(df_furthest.head(3))

      Target Distance_km  Duration(hour+minutes)  Longitude   latitude
1  Amsterdam       4,533                 2883.48   4.904139  52.367573
3      Basel       4,093                 2642.45   7.588576  47.559599
2   Valletta       3,793                 3050.93  14.514100  35.899237
